In [1]:
import itertools
from typing import (Any, Dict, FrozenSet, Hashable, ItemsView, Iterator,
                    KeysView, List, Mapping, NamedTuple, Optional,
                    Sequence, Tuple, Union, ValuesView, cast)

import numpy as np

from cirq import ops, abc, value, extension, linalg, circuits
import cirq

from cirq.contrib import paulistring

In [2]:
def ops_to_matrix(*ops):
    c = cirq.Circuit.from_ops(*ops)
    return c.to_unitary_matrix()
q0 = cirq.NamedQubit('q0')
q1 = cirq.NamedQubit('q1')
q2 = cirq.NamedQubit('q2')

In [3]:
c = cirq.Circuit.from_ops(
    cirq.X(q0),
    cirq.CZ(q0, q1),
    cirq.MeasurementGate('m')(q0, q1),
)
c

q0: ───X───@───M('m')───
           │   │
q1: ───────@───M────────

In [4]:
def cz_count(circuit):
    return sum(isinstance(op, cirq.GateOperation) and isinstance(op, cirq.Rot11Gate)
               for op in circuit)

cc =  paulistring.optimized_circuit(c)
cc

q0: ───@───X───M('m')───
       │       │
q1: ───@───Z───M────────

In [5]:
cirq.google.optimized_for_xmon(cc)

q0: ───@───!M('m')───
       │   │
q1: ───@───M─────────

In [6]:
cirq.testing.assert_allclose_up_to_global_phase(
    c.to_unitary_matrix(),
    cc.to_unitary_matrix(),
    atol=1e-7
)

In [7]:
c_orig = cirq.Circuit.from_ops(
        ops.X(q1) ** 0.5,
        ops.CZ(q0, q1),
        ops.Z(q0) ** 0.25,
        ops.X(q1) ** 0.25,
        ops.CZ(q0, q1),
        ops.X(q1) ** -0.5,
        #ops.MeasurementGate('m1')(q1),
    )
c_orig

q0: ───────────@───T────────@────────────
               │            │
q1: ───X^0.5───@───X^0.25───@───X^-0.5───

In [8]:
paulistring.clifford_optimized_circuit(c_orig)

q0: ───@───[Z]^0.25───@───
       │              │
q1: ───@───[X]^0.25───@───

In [9]:
paulistring.clifford_optimized_circuit(paulistring.pauli_string_optimized_circuit(c_orig))

q0: ───@──────────────@───[Z]^0.25───
       │              │
q1: ───@───[X]^0.25───@──────────────